### 1. 加载数据

In [77]:
import pandas as pd
import numpy as np
import calendar
import datetime

In [3]:
df = pd.read_csv("./data/filter_time_data.txt_new", sep="\t\t", header=None)

/home/wujiaocan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [6]:
df.columns = ['电影ID', '时间', '当前热度值', '新增热度值', '当前点击量', '新增点击量', '当前评论数', '新增评论数', '当前点赞数', '新增点赞数',
              '当前踩数', '新增踩数', '当前评分数', '推广位', '影片推广位等级', '当前小时弹幕数', '当前小时新增弹幕数', '当前小时评分人数',
              '当前小时新增评分人数', '当前小时五星评分人数', '当前小时四星评分人数', '当前小时三星评分人数', '当前小时二星评分人数',
              '当前小时一星评分人数', '当日爱奇艺播放指数', '影片上线时间', '导演', '编剧', '制片', '美术', '主演']

In [4]:
df.shape

(959866, 31)

### 2. 数据探索，补全缺失值

In [25]:
# df.isnull().sum().sort_values(ascending=False)

##### 当前热度值，用后一个填充；
##### 当前小时弹幕数、当前小时新增弹幕数，删除
##### 当前小时评分人数，用后一个填充；当前小时新增评分人数，删除
##### 当日爱奇艺播放指数，用后一个填充

In [65]:
df['当前热度值'].value_counts()[-111111111.0]

1

In [57]:
# 当前小时弹幕数, '当前小时新增弹幕数'
419649 / 959866

0.4371954001912767

In [61]:
# 当前小时评分人数, '当前小时新增评分人数'
274347 / 959866

0.28581802043201865

In [64]:
# 当日爱奇艺播放指数
132214 / 959866

0.1377421431741514

In [52]:
for col in df.columns:
    if -111111111.0 in df[col].unique():
        print(col)

当前热度值
当前小时弹幕数
当前小时新增弹幕数
当前小时评分人数
当前小时新增评分人数
当前小时五星评分人数
当前小时四星评分人数
当前小时三星评分人数
当前小时二星评分人数
当前小时一星评分人数
当日爱奇艺播放指数


In [67]:
df['当前热度值'].replace(-111111111.0, np.nan, inplace=True)
df['当前热度值'] = df['当前热度值'].fillna(method="bfill")

In [69]:
df['当前小时评分人数'].replace(-111111111.0, np.nan, inplace=True)
df['当前小时评分人数'] = df['当前小时评分人数'].fillna(method="bfill")

In [71]:
df['当日爱奇艺播放指数'].replace(-111111111.0, np.nan, inplace=True)
df['当日爱奇艺播放指数'] = df['当日爱奇艺播放指数'].fillna(method="bfill")

### 3. 增加特征

#### 3.1 增加“time_interval”

In [18]:
def get_time_interval(x, y):
    '''
        获取影片已经上映多久；x:现在时间； y:影片上映时间；
    '''
    from dateutil.parser import parse
    a = parse(x)
    b = parse(y)
    return (a - b).total_seconds()

In [19]:
df['time_interval'] = df.apply(
        lambda row: get_time_interval(row['时间'], row['影片上线时间']), axis=1)

#### 3.2 高频导演/主演名，与第一次获取的top10的导演名不同，按本次获取的顺序添加新特征

In [1]:
#  top_10_directors = ['巴晨旭', '郭靖', '朱锐斌', '潘文杰', '薛少', '褚会林', '唐顺风', '干志文', '房映华', '崔俊杰']
#  top_8_actors = ['张伟', '张浩', '李伟', '刘洋', '王伟', '张涛', '张磊', '冯芷墨']

In [29]:
def get_each_movie_relevant_name(data, name_type):
    '''
        按电影ID去重后，获取每所有影片的导演/主演总名单
    '''
    relevant_name = []
    cleaned_relevant_name = []

    # 获取每部影片的导演/主演名字，一个或若干个
    for movie_id in data['电影ID'].unique():
        relevant_name.extend(data[data['电影ID'] == movie_id][name_type].unique().tolist())
    # 将某部影片的若干个导演/主演名字，拆开成一个一个
    for name_pair in relevant_name:
        if name_pair:
            if len(str(name_pair).split(",")) > 1:
                cleaned_relevant_name.extend(name_pair.split(","))
            else:
                cleaned_relevant_name.append(name_pair)

    return cleaned_relevant_name

In [30]:
director_names = get_each_movie_relevant_name(df, "导演")
actor_names = get_each_movie_relevant_name(df, "主演")

In [37]:
top_10_directors = pd.Series(director_names).value_counts()[:10].index.tolist()
top_8_actors = pd.Series(actor_names).value_counts()[:8].index.tolist()

In [40]:
for name in top_10_directors + top_8_actors:
    df[name] = 0

In [44]:
combine_name_number = {}
number = -18  # -18取决现在dataframe的列的情况
for name in top_10_directors + top_8_actors:
    combine_name_number[name] = number
    number += 1

In [45]:
def transfer0to1(data, name_type, name_list, dic):
    '''
        将新增特征列中，将属于高频导演/高频主演/推广位的特征，从0转成1
        "name_type":“导演”或“主演”或“”推广位
        "name_list": 高频导演/高频主演/推广位的总名单
        "dic": 新特征列名与数字索引对应的字典
    '''
    for num, item in enumerate(data[name_type]):

        if item and len(str(item).split(",")) == 1:
            if item in name_list:
                data.iloc[num, dic[item]] = 1

        if item and len(str(item).split(",")) > 1:
            for n in item.split(","):
                if n in name_list:
                    data.iloc[num, dic[n]] = 1
                else:
                    continue

In [46]:
transfer0to1(df, "导演", top_10_directors + top_8_actors, combine_name_number)
transfer0to1(df, "主演", top_10_directors + top_8_actors, combine_name_number)

#### 3.3 “is_weekday”判断是否是工作日

In [74]:
def get_weekday(x):
    '''
        将日期转化为对应的星期数
    '''
    from datetime import datetime
    dateString = x.split()[0]
    week_day = datetime.strptime(dateString, "%Y-%m-%d").weekday()
    return calendar.day_name[week_day]

In [75]:
def is_weekday(day):
    '''
    判断是否是工作日,是为1，不是为0。
    '''
    if day == "Saturday" or day == "Sunday":
        return 0
    else:
        return 1

In [78]:
df['weekday'] = df['时间'].apply(get_weekday)
df['is_weekday'] = df['weekday'].apply(is_weekday)

#### 3.4. “推广位”进行one-hot编码，从前一天晚上10点多跑到第二天早上10点多，不知为何这么久。

In [81]:
def get_each_movie_promotion(data):
    promotion = []
    for pair in data['推广位'].unique().tolist():
        if len(pair.split(",")) > 1:
            promotion.extend(pair.split(","))
        else:
            promotion.append(pair)
    return set(promotion)

In [105]:
get_each_movie_promotion(df)

{'VIP-焦点', 'VIP-电影', '无', '电影-焦点', '电影-电影', '电影-网大', '精选-电影'}

In [106]:
promotion_type = get_each_movie_promotion(df)

In [135]:
for name in promotion_type:
    df[name] = 0

In [130]:
combine_name_number_2 = {}
number = -len(promotion_type)
for name in promotion_type:
    combine_name_number_2[name] = number
    number += 1

In [138]:
transfer0to1(df, "推广位", promotion_type, combine_name_number_2)

In [155]:
# df.columns

In [154]:
# df['电影-焦点'].unique()

In [153]:
# df['推广位'][946195:946395]

In [152]:
# df.iloc[946195:946395, -10:]

#### 2.5 “week_day”进行one-hot编码，具体到每周第几天，训练特征时，直接用get_dummies，比transfer0to1快很多，如果需要预测新样本时，再transfer0to1

In [173]:
for col in pd.get_dummies(df['weekday']).columns:
    df[col] = pd.get_dummies(df['weekday'])[col]

#### 2.6“推广位等级”进行one-hot编码，推广位等级分类

In [177]:
for col in pd.get_dummies(df['影片推广位等级']).columns:
    df[col] = pd.get_dummies(df['影片推广位等级'])[col]

#### 2.7 时间区间特征

In [180]:
def get_hour(x):
    '''
        用“时间”的时间戳补充新特征小时"hour"
    '''
    return x.split()[1].split(":")[0]

In [181]:
def get_time_partition(x):
    '''
        将时间点转化为对应的时间区间
    '''
    x = int(x)
    if x in range(1, 11):   # 1:00-10:00
        return 0
    if x in range(11, 21):  # 11:00-20:00
        return 1
    else:                   # 21:00-0:00
        return 2

In [182]:
df["hour"] = df["时间"].apply(get_hour)
df['time_partition'] = df['hour'].apply(get_time_partition)

In [185]:
for col in pd.get_dummies(df['time_partition']).columns:
    df[col] = pd.get_dummies(df['time_partition'])[col]

#### 2.8 “hour”进行one-hot编码

In [188]:
for col in pd.get_dummies(df['hour']).columns:
    df[col] = pd.get_dummies(df['hour'])[col]

#### 2.9  以6为时间窗口，获取每6条“当前热度值”的平均值

In [193]:
def MaxMinNormalization(x):
    '''
        "当前热度值"归一化处理，0为该列min,6304为该列max
    '''

    x = (float(x) - 0) / (6304 - 0)
    return x

In [194]:
df['当前热度值'] = df['当前热度值'].apply(MaxMinNormalization)

In [196]:
if df.shape[0] < 6:
    df['heat_value_window_size_6'] = df['当前热度值']
else:
    new_now_heat_value = []
    for i in range(df.shape[0]):
        new_now_heat_value.append(df.iloc[i:i + 6, :]['当前热度值'].mean())
    for i in range(5):
        new_now_heat_value.insert(i, 0)
    df['heat_value_window_size_6'] = pd.DataFrame(new_now_heat_value)

### 3. 删除无用特征，最后得到76列特征

In [201]:
# df.to_csv("./data/all_features_df.csv")

In [203]:
del df['电影ID']

# 2. 删除缺失值最多的一列“美术”
del df['美术']

# 3. 删除已经提取出高频名字的“导演”，“主演”
del df['导演']
del df['主演']

# 4. 删除“影片上线时间”，“编剧”，“制片”
del df['影片上线时间']
del df['编剧']
del df['制片']

# 5. 删除用0替换“-111111111.0”之后，0项占比过大的“当前弹幕数”和“当前小时新增弹幕数”
del df['当前小时弹幕数']
del df['当前小时新增弹幕数']

# 6. “当前小时评分人数”，用后一位填充；删除“当前小时新增评分人数”
# original_df['当前小时评分人数'].replace(-111111111.0, np.nan, inplace=True)
# original_df['当前小时评分人数'] = original_df['当前小时评分人数'].fillna(method="bfill")
del df['当前小时新增评分人数']

# 7. 删除'当前小时一星评分人数'，'当前小时二星评分人数'，'当前小时三星评分人数'，'当前小时四星评分人数'，'当前小时五星评分人数'
del df['当前小时一星评分人数']
del df['当前小时二星评分人数']
del df['当前小时三星评分人数']
del df['当前小时四星评分人数']
del df['当前小时五星评分人数']

# 8. 删除以转成one-hot编码的离散特征
del df['时间']
del df['weekday']
del df['hour']
del df['影片推广位等级']
del df['推广位']
del df['time_partition']

# 9. 删除label
del df['当前点击量']
del df['新增点击量']

In [208]:
# df.to_csv("./data/cleaned_df.csv")